<a href="https://colab.research.google.com/github/Kishore-C-Dev/ML-AI/blob/main/summerize_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:



#!pip install --upgrade gradio
#!pip install -U langchain-community
#!pip install pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.6 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import torch
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from PyPDF2 import PdfReader



In [ ]:
device = 0 if torch.cuda.is_available() else -1

In [ ]:
# Load Hugging Face summarization model
hf_summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)

# Wrap the Hugging Face pipeline in a LangChain LLM
llm = HuggingFacePipeline(pipeline=hf_summarizer)




Device set to use cuda:0
<ipython-input-30-52902a14eb5f>:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_summarizer)


In [ ]:
# LangChain prompt template for summarization
prompt_template = PromptTemplate(
    input_variables=["text"],
    template="Provide a concise and comprehensive summary of the following text. Capture the key points, main arguments, and essential information and write in markdown format:\n\n{text}\n\nSummary:"
)
# LangChain summarization chain
summarization_chain = LLMChain(llm=llm, prompt=prompt_template)

# Function to extract text from a PDF
def extract_text_from_pdf(file):
    reader = PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Function to summarize extracted text
def summarize_with_langchain(text, max_chunk_size=2000):
    # Split text into manageable chunks
    chunks = [text[i:i+max_chunk_size] for i in range(0, len(text), max_chunk_size)]

    # Aggregate summaries
    full_summary = ""
    for chunk in chunks:
        result = summarization_chain.run({"text": chunk})
        full_summary += result + "\n\n"

    return full_summary

# Gradio function to handle file upload and summarization
def process_pdf(file):
    # Extract text from the uploaded PDF
    text = extract_text_from_pdf(file)

    # Summarize the text
    summary = summarize_with_langchain(text)
    return summary





In [ ]:
# Create Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### PDF Summarizer Using LangChain with Hugging Face")
    gr.Markdown("Upload a PDF to extract and summarize its content using LangChain and a Hugging Face model.")

    # Input and output components
    file_input = gr.File(label="Upload PDF", file_types=[".pdf"])
    summary_output = gr.Textbox(label="Summary", lines=20)

    # Button to trigger summarization
    summarize_button = gr.Button("Summarize")

    # Define workflow
    summarize_button.click(
        process_pdf,
        inputs=file_input,
        outputs=summary_output,
        api_name="summarize"
    )


In [ ]:
# Launch the Gradio app
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ecbaf7723f04ae640a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
